In [20]:
import os
import torch
import cv2 as cv
import numpy as np
import torchvision
from torchvision import transforms
import pandas as pd

In [21]:
class LateFusionNet(torch.nn.Module):
    def __init__(self, num_classes):
        super(LateFusionNet, self).__init__()
        # Separate convolutional layers for RGB and depth streams
        self.conv_rgb = torchvision.models.resnet18(pretrained=False)
        self.conv_rgb.fc = torch.nn.Identity()
        
        self.conv_depth = torchvision.models.resnet18(pretrained=False)
        self.conv_depth.conv1 = torch.nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3, bias=False)
        self.conv_depth.fc = torch.nn.Identity()
        
        # Fully connected layers for fusion
        self.fc1 = torch.nn.Linear(512 * 2, 128)  # Concatenating features from both streams
        self.fc2 = torch.nn.Linear(128, num_classes)  # Adjust num_classes based on your task
        
    def forward(self, rgbd_inputs):
        rgb_inputs = rgbd_inputs[:,:3,:,:]
        depth_inputs = rgbd_inputs[:,3,:,:].unsqueeze(1)

        # Forward pass for RGB stream
        rgb_features = self.conv_rgb(rgb_inputs)
        # Forward pass for depth stream
        depth_features = self.conv_depth(depth_inputs)
        
        # Concatenate features from both streams
        combined_features = torch.cat((rgb_features, depth_features), dim=1)
        
        # Fully connected layers for fusion
        x = self.fc1(combined_features)
        x = torch.relu(x)
        x = self.fc2(x)
        return x


In [22]:
def model_definition(weights, classes, case):
    
    model = torchvision.models.resnet18(pretrained=False)
    model.fc = torch.nn.Linear(512, classes)

    if case.split('_')[0] == 'depth':
        model.conv1 = torch.nn.Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)

    # elif case.split('_')[0] == 'rgb':
    # model.conv1 = torch.nn.Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
        
    elif case.split('_')[0] == 'Ergbd':
        model.conv1 = torch.nn.Conv2d(4, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)

    elif case.split('_')[0] == 'Lrgbd':
        model = LateFusionNet(classes)
        
    model.eval()
    model.load_state_dict(torch.load(weights)) # Load weights
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    model.to(device)

    return model, device

def im2tens(rgb, depth, case):
        
        rgb_transform = torchvision.transforms.ToTensor()
        depth_transform = torchvision.transforms.Compose([transforms.ToTensor(), transforms.Grayscale(num_output_channels=1)])
                 
        rgb_tensor = rgb_transform(rgb)
        depth_tensor = depth_transform(depth)

        rgbd_tensor = torch.cat([rgb_tensor, depth_tensor], dim=0)
        
        if case.split('_')[0] == 'rgb':
                img_tensor = rgb_tensor
                
        elif case.split('_')[0] == 'depth':
                img_tensor = depth_tensor
        else:
                img_tensor = rgbd_tensor

        return img_tensor
    
def predict(img, model, device):
    
    xb = img.unsqueeze(0)
    yb = model(xb.to(device))
    # Get the normalized probability vector
    p = torch.nn.functional.softmax(yb, dim=1)
    dist = p[0].cpu().detach().numpy()
    
    return dist

def get_models(rad_weights, angle_weights, case):
    
    rad_model, device = model_definition(rad_weights, 6, case)
    angle_model, _     = model_definition(angle_weights, 16, case)

    return rad_model, angle_model, device

def angle_avg_pred(pred, th=0.01):
    
    pred = np.where(pred < th, 0, pred)
    pred = pred/np.sum(pred)
    step = 2*np.pi / len(pred)
    mid_slice = [step/2 + step * interval for interval in range(len(pred))]
    
    sin = np.zeros((len(mid_slice)))
    cos = np.zeros((len(mid_slice)))
    for ii in range(len(mid_slice)):
        sin[ii] = np.sin(mid_slice[ii])
        cos[ii] = np.cos(mid_slice[ii])
        
    dot_sin = np.dot(sin, pred)
    dot_cos = np.dot(cos, pred)
    tan = np.arctan2(dot_sin ,dot_cos)
    avg_pred = np.rad2deg(tan)
    if avg_pred < 0: avg_pred = avg_pred+360
        
    return avg_pred

def rad_avg_pred(pred, th=0.05):
    
    pred = np.where(pred < th, 0, pred)
    pred = pred/np.sum(pred)
    step = np.linspace(0, 6, 6)
    avg_pred = np.dot(step, pred) 
        
    return avg_pred

def main_pred(rgb_img, depth_img, rad_model, angle_model, device, case):
    
    tensor_img = im2tens(rgb_img, depth_img, case)
    rad_dist = predict(tensor_img, rad_model, device)
    angle_dist = predict(tensor_img, angle_model, device)
    rad_pred = rad_avg_pred(rad_dist)
    angle_pred = angle_avg_pred(angle_dist)

    
    return rad_pred, angle_pred

In [23]:
def get_path(folder, case, ver):
    
    if case.split('_')[1] == 'angle':
        rgb_data_dir = os.path.join(folder, 'dataset', 'test', 'rgb')
        depth_data_dir = os.path.join(folder, 'dataset', 'test', 'depth')
        
    elif case.split('_')[1] == 'rad':
        rgb_data_dir = os.path.join(folder, 'dataset', 'test', 'rgb')
        depth_data_dir = os.path.join(folder, 'dataset', 'test', 'depth')
        
    csv_path = os.path.join(folder, 'dataset', 'test', 'test_data.csv')
    
        

    rad_weights  = os.path.join(folder, 'Weights', f"{case.split('_')[0]}_rad_v{ver}.pth")
    angle_weights = os.path.join(folder, 'Weights', f"{case.split('_')[0]}_angle_v{ver}.pth")
    
    output_dir = os.path.join(folder, 'results', f"{case.split('_')[0]}_v{ver}")

    return rgb_data_dir, depth_data_dir, csv_path, rad_weights, angle_weights, output_dir

In [24]:
cases = ['rgb_angle', 'rgb_rad', 'depth_angle', 'depth_rad', 'Ergbd_angle', 'Ergbd_rad', 'Lrgbd_angle', 'Lrgbd_rad']
cameras = ['405', '515']
ver = 0

for camera in cameras:
    print('------------------------------',camera,'------------------------------')
    
    main_folder = os.path.join('.\\', camera)
        
    for folder in os.listdir(main_folder):
        print('--------------------',folder,'--------------------')
        
        folder = os.path.join(main_folder, folder)
        
        for case in cases:
            print('----------',case,'----------')
            
            rgb_img_dir, depth_img_dir, csv_path, rad_weights, angle_weights, output_dir = get_path(folder, case, ver)
            rad_model, angle_model, device = get_models(rad_weights, angle_weights, case)
            
            if not os.path.exists(output_dir):
                os.makedirs(output_dir)
                
            output_csv_path = os.path.join(output_dir, 'graphs_data.csv')

            df = pd.read_csv(csv_path)
            im_name = df.loc[:, 'Picture name']
            angle = df.loc[:, 'Angle [deg]']
            rad = df.loc[:, 'Radial distance [mm]']
            
            data = []
            
            for ii in range(len(im_name)):
                os.path.join(rgb_img_dir, im_name[ii])
                rgb_img = cv.cvtColor(cv.imread(os.path.join(rgb_img_dir, im_name[ii])), cv.COLOR_BGR2RGB)
                depth_img = cv.cvtColor(cv.imread(os.path.join(depth_img_dir, im_name[ii])), cv.COLOR_BGR2RGB)
                rad_pred, angle_pred = main_pred(rgb_img, depth_img, rad_model, angle_model, device, case)    
                    
                if np.abs(angle_pred - angle[ii]) > 180:
                    angle_pred = 360 - angle_pred
                    
                data.append({
                    'Picture name': im_name[ii],
                    'gt_rad': rad[ii],
                    'pred_rad': rad_pred,
                    'gt_angle': angle[ii],
                    'pred_angle': angle_pred
                })
                
            # Convert the list of dictionaries to a DataFrame
            data_df = pd.DataFrame(data)

            # Save the DataFrame to a CSV file
            data_df.to_csv(output_csv_path, index=False)
                

------------------------------ 405 ------------------------------
-------------------- Medical --------------------
---------- rgb_angle ----------
---------- rgb_rad ----------
---------- depth_angle ----------
---------- depth_rad ----------
---------- Ergbd_angle ----------
---------- Ergbd_rad ----------
---------- Lrgbd_angle ----------
---------- Lrgbd_rad ----------
-------------------- PIH --------------------
---------- rgb_angle ----------
---------- rgb_rad ----------
---------- depth_angle ----------
---------- depth_rad ----------
---------- Ergbd_angle ----------
---------- Ergbd_rad ----------
---------- Lrgbd_angle ----------
---------- Lrgbd_rad ----------
------------------------------ 515 ------------------------------
-------------------- Medical --------------------
---------- rgb_angle ----------
---------- rgb_rad ----------
---------- depth_angle ----------
---------- depth_rad ----------
---------- Ergbd_angle ----------
---------- Ergbd_rad ----------
--------